In [ ]:
! pip install -q boto3==1.34.71 langchain==0.1.13

In [ ]:
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
import os
from langchain.chains import ConversationChain
from langchain.llms.bedrock import Bedrock
from langchain.memory import ConversationBufferMemory
import boto3

load_dotenv(".env", override=True)

# Currently, bedrock-runtime is only available in a few regions.

aws_access_key_id = os.environ['AWS_BEDROCK_PUBLIC_KEY']
aws_secret_access_key = os.environ['AWS_BEDROCK_SECRET_KEY']

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

session = boto3.session.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name="us-east-1"
)

# Use the session to create the bedrock-runtime client
boto3_bedrock = session.client('bedrock-runtime')
modelId = "anthropic.claude-v2"

cl_llm = Bedrock(
    model_id=modelId,
    client=boto3_bedrock,
    model_kwargs={"max_tokens_to_sample": 1000},
)
memory = ConversationBufferMemory()

claude_prompt = PromptTemplate.from_template("""

Human: The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context. If the AI does not know
the answer to a question, it truthfully says it does not know.

Current conversation:
<conversation_history>
{history}
</conversation_history>

Here is the human's next reply:
<human_reply>
{input}
</human_reply>

Assistant:
""")

# Turn verbose to true to see the full logs and documents
conversation= ConversationChain(
    llm=cl_llm, 
    verbose=False, 
    prompt=claude_prompt,
    memory=memory
)

# conversation.prompt = claude_prompt

result = conversation.predict(input="Hi there!")

print(result)